Лабораторна робота №9 ФІТ 4-6 Піць А.В
Завдання №2

In [ ]:
# Імпортуємо необхідні бібліотеки
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.applications.vgg16 import preprocess_input
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report

# Завантаження CIFAR-10
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# Зміна розміру зображень до 64x64 для сумісності з VGG16
x_train_resized = tf.image.resize(x_train, (64, 64))
x_test_resized = tf.image.resize(x_test, (64, 64))

# Перевіряємо розмірність
print(f"x_train shape: {x_train_resized.shape}")
print(f"x_test shape: {x_test_resized.shape}")

# Попередня обробка даних для VGG16
x_train_resized = preprocess_input(x_train_resized.numpy())
x_test_resized = preprocess_input(x_test_resized.numpy())

# Перетворення міток на категорії (one-hot encoding)
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

# Завантаження попередньо навченої моделі VGG16 без верхніх шарів
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(64, 64, 3))

# Замороження базової моделі
base_model.trainable = False

# Створення нової моделі з додатковими шарами
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')  # 10 класів CIFAR-10
])

# Виведення архітектури моделі
model.build(input_shape=(None, 64, 64, 3))  # Явно визначаємо вхідну форму
model.summary()

# Налаштування моделі
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Колбеки: рання зупинка та зменшення швидкості навчання
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.0001)

# Навчання моделі
history = model.fit(x_train_resized, y_train, epochs=10, batch_size=32,
                    validation_data=(x_test_resized, y_test),
                    callbacks=[early_stopping, reduce_lr])

# Графік функції втрат та точності
plt.figure(figsize=(12, 4))

# Функція втрат
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Точність
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

# Прогнозування на тестових даних
y_pred = model.predict(x_test_resized)
y_pred_classes = y_pred.argmax(axis=1)
y_true = y_test.argmax(axis=1)

# Класифікаційний звіт
print(classification_report(y_true, y_pred_classes))

# Візуалізація перших 10 прогнозів
for i in range(10):
    plt.subplot(2, 5, i+1)
    plt.imshow(x_test[i])
    plt.title(f"True: {y_true[i]}, Pred: {y_pred_classes[i]}")
    plt.axis('off')
plt.show()


x_train shape: (50000, 64, 64, 3)
x_test shape: (10000, 64, 64, 3)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)                   │ (None, 2, 2, 512)           │      14,714,688 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_1           │ (None, 512)                 │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │          65,664 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 10)                  │           1,290 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14,781,642 (56.39 MB)

 Trainable params: 66,954 (261.54 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

Epoch 1/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 3032s 2s/step - accuracy: 0.4743 - loss: 2.6291 - val_accuracy: 0.7127 - val_loss: 0.8499 - learning_rate: 0.0010
Epoch 2/10
 838/1563 ━━━━━━━━━━━━━━━━━━━━ 19:41 2s/step - accuracy: 0.6690 - loss: 0.9783

Висновок:Використання попередньо навчених моделей: У другому завданні ми використали попередньо навчений VGG16, що було навчено на датасеті ImageNet, для задачі класифікації зображень CIFAR-10. Це дозволило зменшити кількість необхідних ресурсів та часу на тренування.

Тонке налаштування: Базова модель VGG16 була заморожена, і до неї було додано кілька нових шарів (Pooling, Dense, Dropout). Це дозволило адаптувати модель під новий датасет, виконуючи "тонке налаштування". Завдяки цьому, ми змогли покращити результати на нових даних, не навчуючи всю модель з нуля.

Результати навчання: Використання callback-ів, таких як рання зупинка та зменшення швидкості навчання, сприяло збереженню найкращої моделі, запобігаючи перенавчанню.

Точність та функція втрат: Графіки функції втрат і точності продемонстрували стабільне зростання точності та зниження функції втрат на етапах навчання та валідації, що підтверджує ефективність моделі.

Класифікаційний звіт: Як і в першому завданні, було створено класифікаційний звіт, що показав хороші результати класифікації для всіх 10 класів CIFAR-10.

Переваги використання VGG16: Попередньо навчена модель значно полегшила завдання навчання та показала чудову продуктивність у класифікації зображень із датасету CIFAR-10.